# 거래소(대신증권) 에서 kospi, kosdaq 종목 코드 뽑아내기
* 변수
    * codes = {corpName: stockCode}

In [6]:
import win32com.client

instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
codeList = instCpCodeMgr.GetStockListByMarket(1) # 코스피
codeList2 = instCpCodeMgr.GetStockListByMarket(2) # 코스닥

In [6]:
j = 0
for i, code in enumerate(codeList2):
    if code[-1] == '0':
        secondCode = instCpCodeMgr.GetStockSectionKind(code)
        if secondCode == 1:
            name = instCpCodeMgr.CodeToName(code)
            price = objCpCodeMgr.GetStockYdClosePrice(code)
            print(j, code, secondCode, name, price)
            j += 1

0 A000250 1 삼천당제약 45200
1 A000440 1 중앙에너비스 15850
2 A001000 1 신라섬유 2235
3 A001540 1 안국약품 9930
4 A001810 1 무림SP 3105
5 A001840 1 이화공영 5740
6 A002230 1 피에스텍 5060
7 A002290 1 삼일기업공사 3995
8 A002680 1 한탑 1315
9 A002800 1 신신제약 6680
10 A003100 1 선광 54500
11 A003310 1 대주산업 2195
12 A003380 1 하림지주 9850
13 A003800 1 에이스침대 42500
14 A004590 1 한국가구 6650
15 A004650 1 창해에탄올 13650
16 A004780 1 대륙제관 5230
17 A005160 1 동국산업 3685
18 A005290 1 동진쎄미켐 37100
19 A005670 1 푸드웰 6550
20 A005710 1 대원산업 6910
21 A005860 1 한일사료 2000
22 A005990 1 매일홀딩스 9290
23 A006050 1 국영지앤엠 2155
24 A006140 1 피제이전자 8190
25 A006580 1 대양제지 3260
26 A006620 1 동구바이오제약 7700
27 A006730 1 서부T&D 8990
28 A006910 1 보성파워텍 3680
29 A006920 1 모헨즈 5420
30 A007330 1 푸른저축은행 12750
31 A007370 1 진양제약 6770
32 A007390 1 네이처셀 17700
33 A007530 1 영신금속 2735
34 A007680 1 대원 9790
35 A007720 1 대명소노시즌 1270
36 A007770 1 한일화학 16600
37 A007820 1 에스엠코어 7040
38 A008290 1 원풍물산 1520
39 A008370 1 원풍 5170
40 A008470 1 부스타 5880
41 A008830 1 대동기어 5930
42 A009300 1 삼아제약 18700
4

# DART 에서 종목코드-회사코드 쌍 가져오기
* 변수
    * info = {stockCode: corpCode, corpName}

In [12]:
import requests
from zipfile import ZipFile
from io import BytesIO
import xml.etree.ElementTree as et
import pandas as pd

In [13]:
url = 'https://opendart.fss.or.kr/api/corpCode.xml'
key = '6e2b668b16e3457e032a8c7c03f5ee3f7af9d221'
params = {'crtfc_key': key}
res = requests.get(url, params)

In [14]:
zo = ZipFile(BytesIO(res.content))

xmlfile_objs = {name: zo.read(name) for name in zo.namelist()}
xml_str = xmlfile_objs['CORPCODE.xml'].decode('utf-8')

In [15]:
# XML String을 가져와서 DataFrame에 담기
xroot = et.fromstring(xml_str)

df_cols = ["corp_code", "corp_name", "stock_code", "modify_date"]
rows = []

for node in xroot: 
    res = []
    for el in df_cols[0:]: 
        if node is not None and node.find(el) is not None:
            res.append(node.find(el).text)
        else: 
            res.append(None)
    rows.append({df_cols[i]: res[i] 
                  for i, _ in enumerate(df_cols)})

df = pd.DataFrame(rows, columns=df_cols)


In [16]:
stock = df[df['stock_code'] != ' ']
stock
stock.to_csv('codes.csv')

In [17]:
ccode = df[df['stock_code']=='002680']['corp_code']
ccode

62399    00140566
Name: corp_code, dtype: object

# DART 에서 회사 코드로 재무제표 가져오기
* 리스트 원소 하나하나가 전부 row 임.
* 단일 회사 전체 재무제표를 봐야 일반적인 모든 정보를 볼 수 있음.
    * 재무상태표 + 손익계산서 + 현금흐름표 다 있음
* 최종적으로 사용할 가공 데이터
    * df = corpName | corpCode | stockCode | 당기 순이익 | 유동자산? | 채무 | 주가 | 시총 | PBR | 모멘텀 | 등등

In [18]:
url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
key = '6e2b668b16e3457e032a8c7c03f5ee3f7af9d221'
params = {'crtfc_key': key, 'corp_code': ccode, 'bsns_year': '2021', 'reprt_code': '11013', 'fs_div': 'CFS'}
res = requests.get(url, params)

In [20]:
j

{'status': '000',
 'message': '정상',
 'list': [{'rcept_no': '20210517001890',
   'reprt_code': '11013',
   'bsns_year': '2021',
   'corp_code': '00140566',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'account_id': 'ifrs-full_CurrentAssets',
   'account_nm': '유동자산',
   'account_detail': '-',
   'thstrm_nm': '제 63 기 1분기말',
   'thstrm_amount': '36624282961',
   'frmtrm_nm': '제 62 기말',
   'frmtrm_amount': '36232237782',
   'ord': '1',
   'currency': 'KRW'},
  {'rcept_no': '20210517001890',
   'reprt_code': '11013',
   'bsns_year': '2021',
   'corp_code': '00140566',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'account_id': 'ifrs-full_CashAndCashEquivalents',
   'account_nm': '현금및현금성자산',
   'account_detail': '-',
   'thstrm_nm': '제 63 기 1분기말',
   'thstrm_amount': '4022681390',
   'frmtrm_nm': '제 62 기말',
   'frmtrm_amount': '2160830534',
   'ord': '2',
   'currency': 'KRW'},
  {'rcept_no': '20210517001890',
   'reprt_code': '11013',
   'bsns_year': '2021',
   'corp_code': '00140566',
   'sj_div

In [19]:
j = res.json()

[subject['account_nm'] for subject in j['list']]

['유동자산',
 '현금및현금성자산',
 '단기금융상품',
 '기타유동금융자산',
 '매출채권 및 기타유동채권',
 '재고자산',
 '기타유동자산',
 '당기법인세자산',
 '비유동자산',
 '장기금융상품',
 '기타비유동금융자산',
 '종속기업, 조인트벤처와 관계기업에 대한 투자자산',
 '투자부동산',
 '유형자산',
 '영업권 이외의 무형자산',
 '장기매출채권 및 기타비유동채권',
 '이연법인세자산',
 '자산총계',
 '유동부채',
 '매입채무 및 기타유동채무',
 '단기금융부채',
 '당기법인세부채',
 '기타유동부채',
 '비유동부채',
 '장기금융부채',
 '기타비유동부채',
 '이연법인세부채',
 '퇴직급여부채',
 '부채총계',
 '지배기업의 소유주에게 귀속되는 자본',
 '자본금',
 '보통주자본금',
 '기타자본구성요소',
 '이익잉여금(결손금)',
 '비지배지분',
 '자본총계',
 '자본과부채총계',
 '수익(매출액)',
 '매출원가',
 '매출총이익',
 '판매비와관리비',
 '영업이익(손실)',
 '기타이익',
 '기타손실',
 '금융수익',
 '금융원가',
 '관계기업투자이익',
 '관계기업투자손실',
 '법인세비용차감전순이익(손실)',
 '법인세비용',
 '계속영업이익(손실)',
 '당기순이익(손실)',
 '기타포괄손익',
 '당기손익으로 재분류되지 않는항목(세후기타포괄손익)',
 '확정급여제도의 재측정손익(세후기타포괄손익)',
 '재평가차액',
 '당기손익으로 재분류될 수 있는 항목(세후기타포괄손익)',
 '해외사업장환산외환차이(세후기타포괄손익)',
 '매도가능금융자산평가손익(세후기타포괄손익)',
 '총포괄손익',
 '지배기업의 소유주에게 귀속되는 당기순이익(손실)',
 '비지배지분에 귀속되는 당기순이익(손실)',
 '총 포괄손익, 지배기업의 소유주에게 귀속되는 지분',
 '총 포괄손익, 비지배지분',
 '기본주당이익(손실)',
 '희석주당이익(손실)',
 '영업활동현금흐름',
 '영업으로부터 창출된 현금흐름',
 '이자수취(

## naver finalce API (REST) 로 주가 데이터 받아오는 함수

In [8]:
from urllib import parse
from ast import literal_eval
import requests

def get_sise(code, start_time, end_time, time_from='day') :
    get_param = {
    	'symbol':code,
	    'requestType':1,
	    'startTime':start_time,
	    'endTime':end_time,
	    'timeframe':time_from
    }
    get_param = parse.urlencode(get_param)
    url="https://api.finance.naver.com/siseJson.naver?%s"%(get_param)
    response = requests.get(url)
    return literal_eval(response.text.strip())

## 거래소에서 주가 데이터 받아오기 + 전일 종가까지

In [7]:
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    
# 종목코드 리스트 구하기
objCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
codeList = objCpCodeMgr.GetStockListByMarket(1) #거래소
codeList2 = objCpCodeMgr.GetStockListByMarket(2) #코스닥


print("거래소 종목코드", len(codeList))
for i, code in enumerate(codeList):
    secondCode = objCpCodeMgr.GetStockSectionKind(code)
    name = objCpCodeMgr.CodeToName(code)
    stdPrice = objCpCodeMgr.GetStockStdPrice(code)
    print(i, code, secondCode, stdPrice, name)
    
    get_sise(code, '20210101', '20220505', 'month')
# print("코스닥 종목코드", len(codeList2))
# for i, code in enumerate(codeList2):
#     secondCode = objCpCodeMgr.GetStockSectionKind(code)
#     name = objCpCodeMgr.CodeToName(code)
#     stdPrice = objCpCodeMgr.GetStockStdPrice(code)
#     print(i, code, secondCode, stdPrice, name)



거래소 종목코드 1730
0 A000020 1 13450 동화약품
1 A000040 1 905 KR모터스
2 A000050 1 13650 경방
3 A000060 1 31250 메리츠화재
4 A000070 1 99300 삼양홀딩스
5 A000075 1 69700 삼양홀딩스우
6 A000080 1 31850 하이트진로
7 A000087 1 19650 하이트진로2우B
8 A000100 1 59300 유한양행
9 A000105 1 58400 유한양행우
10 A000120 1 131000 CJ대한통운
11 A000140 1 13450 하이트진로홀딩스
12 A000145 1 16000 하이트진로홀딩스우
13 A000150 1 123500 두산
14 A000155 1 77500 두산우
15 A000157 1 115500 두산2우B
16 A000180 1 2415 성창기업지주
17 A000210 1 60300 DL
18 A000215 1 36850 DL우
19 A000220 1 6920 유유제약
20 A000225 1 8730 유유제약1우
21 A000227 1 17000 유유제약2우B
22 A000230 1 14300 일동홀딩스
23 A000240 1 15700 한국앤컴퍼니
24 A000270 1 82400 기아
25 A000300 1 1605 대유플러스
26 A000320 1 12450 노루홀딩스
27 A000325 1 39100 노루홀딩스우
28 A000370 1 3960 한화손해보험
29 A000390 1 9350 삼화페인트
30 A000400 1 1775 롯데손해보험
31 A000430 1 3975 대원강업
32 A000480 1 84300 조선내화
33 A000490 1 9870 대동
34 A000500 1 24600 가온전선
35 A000520 1 7590 삼일제약
36 A000540 1 3710 흥국화재
37 A000545 1 7980 흥국화재우
38 A000547 1 26100 흥국화재2우B
39 A000590 1 61100 CS홀딩스
40 A000640 1

KeyboardInterrupt: 